In [1]:
import numpy as np
import json
import os
import re
import collections
from sklearn.datasets.base import load_files
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
import codecs
import pickle
from gensim.models.word2vec import LineSentence
from gensim.models.fasttext import FastText
import sys
from __future__ import division
from collections import OrderedDict
import pandas as pd
from sklearn.datasets import load_files
from collections import OrderedDict

In [2]:
from surprise import NormalPredictor
from surprise import Dataset
from surprise import Reader
from surprise.model_selection import cross_validate
from surprise.prediction_algorithms.matrix_factorization import SVD
from surprise.model_selection import KFold
from surprise import accuracy

In [3]:
def normalize_name(name):
    name = name.lower()
    name = re.sub(r"[.,\/#!$%\^\*;:{}=\_`~()@]", ' ', name)
    name = re.sub(r'\s+', ' ', name).strip()
    return name

In [4]:
#Change this as per your directory
path = '/home/irlab/Documents/recsys/mpd/data'
filenames = os.listdir(path)
mapping = {}
playlist_prob = OrderedDict()

In [5]:
#Train FastText Model for finding similarity
def train_playlist_embedding(filenames):    
    data = LineSentence('/home/irlab/Documents/recsys/mpd/codes/data.txt')
    model = FastText(size=100)
    print('Building Vocab....')
    model.build_vocab(data)
    print('Training....')
    model.train(data, total_examples=model.corpus_count, epochs=model.iter)
    print(model)
    return model

In [6]:
#print(train_data)
model = train_playlist_embedding(filenames)

Building Vocab....
Training....


/home/irlab/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:8: DeprecationWarning: Call to deprecated `iter` (Attribute will be removed in 4.0.0, use self.epochs instead).
  


FastText(vocab=9132, size=100, alpha=0.025)


In [7]:
#Find Similar Playlists with given name
def find_similar(playlist_name,model = model):
    names = []
    similar_playlists = model.wv.similar_by_word(playlist_name,20)
    similar_playlists = dict(similar_playlists)
    sim_playlist_name = [key for (key,value) in sorted(similar_playlists.items(),key=lambda x:x[1],reverse=True)]
    return sim_playlist_name

In [8]:
#Load mapping of playlist to filename
def load_filemapping():
    fp = open('/home/irlab/Documents/recsys/mpd/codes/mapping/track_filename.json')
    mpd_mapping = json.load(fp)
    return mpd_mapping

In [9]:
mpd_mapping = load_filemapping()

In [10]:
#Load album, artist, track mappings
#Change the directory as per your system

fp = open('/home/irlab/files/stats/album_stats.json')
album_map = json.load(fp)
album_stat = sum(album_map.values())

fp = open('/home/irlab/files/stats/artist_stats.json')
artist_map = json.load(fp)
artist_stat = sum(artist_map.values())

fp = open('/home/irlab/files/stats/tracks_stats.json')
track_map = json.load(fp)
track_stat = sum(track_map.values())

print(album_stat,artist_stat,track_stat)

(66346428, 66346428, 66346428)


In [11]:
#Generate the ratings for all the tracks

def create_weighted_playlists(playlist_name,playlist_prob):
    try:
        similar_playlists = find_similar(playlist_name)
    except KeyError:
        temp = playlist_name.split()
        playlist_name = ''.join(temp)
        similar_playlists = find_similar(playlist_name)
        
    #print(similar_playlists)
    for similar_playlist in similar_playlists:
        if similar_playlist in mpd_mapping.keys():    
            #print(similar_playlist)
            name = mpd_mapping[similar_playlist]
            filename = os.sep.join((path,name))
            fp = open(filename,'r')
            playlist_file = json.load(fp)
            mpd_data = playlist_file['playlists']
            playlist_prob[similar_playlist] = {}
            for playlist in mpd_data:
                if normalize_name(playlist['name']) in similar_playlist:
                    for track in playlist['tracks']:
                        p1 = artist_map[track['artist_uri']]/artist_stat
                        p2 = album_map[track['album_uri']]/album_stat
                        p3 = track_map[track['track_uri']]/track_stat
                        playlist_prob[similar_playlist][track['track_uri']] = (p1+p2+p3)/(1+track['pos'])
                        #print(playlist_prob[similar_playlist])
    return playlist_prob

In [13]:
#Create Rating matrix
def create_matrix(playlist_prob):
    pd.DataFrame(data=playlist_prob,dtype=np.int64)
    df = pd.DataFrame(data=playlist_prob)
    df = df.fillna(0)
    df = df.T
    return df

In [14]:
#Load data in surprise api format
def load_data(tracks,playlists,ratings):
    ratings_dict = {'itemID': tracks,
                    'userID': playlists,
                    'rating': ratings}
    df1 = pd.DataFrame(ratings_dict)
    reader = Reader(rating_scale=(0,1))
    data = Dataset.load_from_df(df1[['userID','itemID','rating']],reader)
    #print(df1)
    return data

In [15]:
#Train data using SVD and gradient descent
def train(data):
    kf = KFold(n_splits=10)
    algo = SVD(n_epochs=60,lr_all=0.007,reg_all=0.075)
    for trainset,testset in kf.split(data):
        algo.fit(trainset)
        prediction = algo.test(testset)
        print(accuracy.rmse(prediction,verbose=True))
    return algo

In [17]:
#Predict ratings for tracks
def test(playlist_name,tracks,algo,seed_tracks):
    tracks_pred = [n for n in tracks if n not in seed_tracks]
    Recommend = {}
    for track in tracks_pred:
        predict = algo.predict(playlist_name,track)
        Recommend[track] = predict[3]
    recommend_list = [key for (key,value) in sorted(Recommend.items(),key=lambda x:x[1],reverse=True)]
    return recommend_list

In [18]:
#Get tracks if recommendations less than 500
def random_tracks(recommend_list,seed_tracks):
    tracks = [key for (key,value) in sorted(track_map.items(),key=lambda x:x[1],reverse=True)]
    extend_tracks = [track for track in tracks if track not in recommend_list]
    extend_tracks = [track for track in extend_tracks if track not in seed_tracks]
    diff = 500 - len(recommend_list)
    print(len(extend_tracks[0:diff]))
    for name in extend_tracks[0:diff]:
        print(name)
        recommend_list.append(name)
    return recommend_list

In [23]:
#Change this as per your directory
path1 = '/home/irlab/Documents/recsys/challenge'
challenge_file = 'challenge_set.json'
filename = os.sep.join((path1,challenge_file))
fp = open(filename)
challenge_set = json.load(fp)
playlists = challenge_set['playlists']
dict_playlist = {}

In [20]:
import csv

In [24]:
#string = ['team_info','main','Dark_Horse_1','201711040@daiict.ac.in']

#with open('/home/irlab/Documents/recsys/mpd/codes/output/output2.csv','w') as csvfile:
#        filewriter = csv.writer(csvfile,delimiter = ',')
#        filewriter.writerow(string)

count=0
for playlist in playlists:
    list_temp = []
    seed_tracks = []
    recommend_list = []
    if 'name' in playlist.keys() and playlist['num_samples']==100:
        pid = playlist['pid']
        list_temp.insert(0,pid)
        count+=1
        print(count,playlist['name'])
        tracks = playlist['tracks']
        playlist_prob.clear()
        playlist_prob[normalize_name(playlist['name'])] = {}
        i=0
        for track in tracks:
            if track['artist_uri'] not in artist_map.keys():
                p1 = 1
            else:    
                p1 = artist_map[track['artist_uri']]/artist_stat
            if track['album_uri'] not in artist_map.keys():
                p2 = 1
            else:
                p2 = album_map[track['album_uri']]/album_stat
            if track['track_uri'] not in artist_map.keys():
                p3 = 1
            else:
                p3 = track_map[track['track_uri']]/track_stat
            seed_tracks.insert(i,track['track_uri'])
            playlist_prob[normalize_name(playlist['name'])][track['track_uri']] = (p1+p2+p3)/(1+track['pos']) 
            i+=1
        #print(seed_tracks)
        print('Finding Similar Playlists...')
        playlist_prob = create_weighted_playlists(normalize_name(playlist['name']),playlist_prob)
        print('Creating Rating matrix...')
        df = create_matrix(playlist_prob)
        column_names = list(df.columns)
        row_names = list(df.index)
        matrix = df.values
        row,col = np.nonzero(matrix)
        row = row.tolist()
        col = col.tolist()
        n_tracks = [column_names[i] for i in col]
        playlists = [row_names[i] for i in row]
        ratings = [matrix[r][c] for (r,c) in zip(row,col)]
        print('Loading data for training')
        data = load_data(n_tracks,playlists,ratings)
        print('Training....')
        algo = train(data)
        print('Predicting....')
        recommend_list = test(normalize_name(playlist['name']),n_tracks,algo,seed_tracks)
        print('Playlist Size:',len(recommend_list))
        if len(recommend_list[0:500])<500:
            print('Picking popular tracks...')
            recommend_list = random_tracks(recommend_list,seed_tracks)
            print('New Playlist Size:',len(recommend_list))
        list_temp.extend(recommend_list[0:500])
        print('Writing to file.....')
        #Change this as per your directory
        with open('/home/irlab/Documents/recsys/mpd/codes/output/final_submission.csv','a') as csvfile1:
            filewriter = csv.writer(csvfile1,delimiter = ',')
            filewriter.writerow(list_temp)

(1, u'April')
Finding Similar Playlists...
Creating Rating matrix...
Loading data for training
Training....
RMSE: 0.0224
0.022423401281603406
RMSE: 0.0674
0.06739761100371881
RMSE: 0.0142
0.014182678872969121
RMSE: 0.0322
0.03216382004457529
RMSE: 0.0168
0.01677920922128298
RMSE: 0.0440
0.043978672485569754
RMSE: 0.0462
0.046187001179481324
RMSE: 0.1394
0.13936877866359224
RMSE: 0.0390
0.039038070249902856
RMSE: 0.0127
0.012749308157710883
Predicting....
('Playlist Size:', 1694)
Writing to file.....
(2, u'Other')
Finding Similar Playlists...
Creating Rating matrix...
Loading data for training
Training....
RMSE: 0.0200
0.02000247961337556
RMSE: 0.1579
0.15794934356176052
RMSE: 0.0354
0.03535034759695963
RMSE: 0.0295
0.029546347963224088
RMSE: 0.0174
0.017371884181481587
RMSE: 0.0202
0.0202328610806596
RMSE: 0.0229
0.02292215172046563
RMSE: 0.0489
0.04886147389946616
RMSE: 0.0764
0.07641209356041934
RMSE: 0.0234
0.02342907372789073
Predicting....
('Playlist Size:', 1344)
Writing to file.

RMSE: 0.0286
0.028570484325615524
RMSE: 0.0109
0.010900479762401402
RMSE: 0.1112
0.11116169330500623
RMSE: 0.0147
0.014729482694059404
RMSE: 0.0203
0.02032625794941989
RMSE: 0.0152
0.015243733037852541
RMSE: 0.0328
0.03277268774965824
RMSE: 0.0289
0.028928622199795796
RMSE: 0.0176
0.017611024174867575
RMSE: 0.0229
0.022857348541864074
Predicting....
('Playlist Size:', 3129)
Writing to file.....
(18, u'Jesus')
Finding Similar Playlists...
Creating Rating matrix...
Loading data for training
Training....
RMSE: 0.0295
0.029470182626674536
RMSE: 0.0626
0.06259107906875361
RMSE: 0.0139
0.013858710994639839
RMSE: 0.0144
0.014409344245368673
RMSE: 0.0163
0.016346864678960713
RMSE: 0.0192
0.01918035016938004
RMSE: 0.0436
0.0435820147121025
RMSE: 0.0125
0.012509176882905636
RMSE: 0.1322
0.13215402856611294
RMSE: 0.0128
0.01275235828071338
Predicting....
('Playlist Size:', 1854)
Writing to file.....
(19, u'Noah')
Finding Similar Playlists...
Creating Rating matrix...
Loading data for training
Tra

RMSE: 0.0145
0.014483617585310303
RMSE: 0.0162
0.016238380397229305
RMSE: 0.0265
0.026493077264111067
RMSE: 0.0233
0.02331477837824644
RMSE: 0.0195
0.01954625280653494
RMSE: 0.0386
0.03856704189938343
RMSE: 0.0183
0.018310294257385215
RMSE: 0.0972
0.09715302342431427
Predicting....
('Playlist Size:', 2466)
Writing to file.....
(34, u'Hype')
Finding Similar Playlists...
Creating Rating matrix...
Loading data for training
Training....
RMSE: 0.0121
0.012134534458562317
RMSE: 0.0118
0.011773579678396469
RMSE: 0.1211
0.12112149403091275
RMSE: 0.0134
0.01337798436451471
RMSE: 0.0189
0.01888517947853943
RMSE: 0.0112
0.011243464404403134
RMSE: 0.0411
0.04114889764528032
RMSE: 0.0236
0.023613441389838917
RMSE: 0.0142
0.014243016959384689
RMSE: 0.0626
0.06259152764486753
Predicting....
('Playlist Size:', 2059)
Writing to file.....
(35, u'PROM 2017')
Finding Similar Playlists...
Creating Rating matrix...
Loading data for training
Training....
RMSE: 0.0116
0.01161152025386677
RMSE: 0.0218
0.021781

RMSE: 0.0395
0.039462005684254876
RMSE: 0.0202
0.020180850962686122
RMSE: 0.0212
0.0212422718278568
RMSE: 0.0143
0.014255140677350755
RMSE: 0.0158
0.01578692674276393
RMSE: 0.0131
0.013054016895420299
RMSE: 0.1272
0.12716962017016412
Predicting....
('Playlist Size:', 2158)
Writing to file.....
(50, u'hoes ')
Finding Similar Playlists...
Creating Rating matrix...
Loading data for training
Training....
RMSE: 0.1637
0.163743272957562
RMSE: 0.0182
0.018168075197904584
RMSE: 0.0088
0.0087983989456633
RMSE: 0.0184
0.018445311858014717
RMSE: 0.0214
0.02142090520526691
RMSE: 0.0203
0.02034280826275416
RMSE: 0.0443
0.04431925168745731
RMSE: 0.0779
0.07790840423442995
RMSE: 0.0204
0.020357609896749812
RMSE: 0.0500
0.049952476919704586
Predicting....
('Playlist Size:', 1293)
Writing to file.....
(51, u'Middle School Jamz')
Finding Similar Playlists...
Creating Rating matrix...
Loading data for training
Training....
RMSE: 0.0154
0.015437102450142424
RMSE: 0.0210
0.021022939695470595
RMSE: 0.0434
0

RMSE: 0.0171
0.017113377883912824
RMSE: 0.0355
0.035465765541497524
RMSE: 0.0148
0.014834609970126377
RMSE: 0.0832
0.08316722625986317
RMSE: 0.0183
0.018328330790456166
RMSE: 0.1508
0.15078893769944055
Predicting....
('Playlist Size:', 1540)
Writing to file.....
(66, u'Mine')
Finding Similar Playlists...
Creating Rating matrix...
Loading data for training
Training....
RMSE: 0.0777
0.07771710495175943
RMSE: 0.0470
0.0469876495236268
RMSE: 0.0088
0.008787184422292213
RMSE: 0.0478
0.047848432597155974
RMSE: 0.0222
0.02219677574149825
RMSE: 0.0138
0.01384306382689214
RMSE: 0.0218
0.02175840292312433
RMSE: 0.0197
0.01970019689430832
RMSE: 0.1559
0.15588105664484506
RMSE: 0.0172
0.017154910419505107
Predicting....
('Playlist Size:', 1347)
Writing to file.....
(67, u'country \U0001f1fa\U0001f1f8')
Finding Similar Playlists...
Creating Rating matrix...
Loading data for training
Training....
RMSE: 0.0481
0.048098492981102935
RMSE: 0.0953
0.09530635429157566
RMSE: 0.0351
0.03513354999747074
RMSE

...

RMSE: 0.0092
0.009247110617006397
RMSE: 0.0153
0.015255921810928117
RMSE: 0.0076
0.0075588354260317115
Predicting....
('Playlist Size:', 3963)
Writing to file.....
(1934, u'Old School Hip Hop')
Finding Similar Playlists...
Creating Rating matrix...
Loading data for training
Training....
RMSE: 0.0126
0.012588403674531939
RMSE: 0.0213
0.021303932140059434
RMSE: 0.0083
0.008254777779239999
RMSE: 0.0076
0.007557906401520769
RMSE: 0.0370
0.03703962260373323
RMSE: 0.0672
0.0672497241247656
RMSE: 0.0157
0.015664208542804516
RMSE: 0.0248
0.024827908235693717
RMSE: 0.0109
0.010939870441213145
RMSE: 0.0150
0.014990969538378834
Predicting....
('Playlist Size:', 1727)
Writing to file.....
(1935, u'gurl')
Finding Similar Playlists...
Creating Rating matrix...
Loading data for training
Training....
RMSE: 0.0156
0.015635951101734775
RMSE: 0.0184
0.018364153219337796
RMSE: 0.0425
0.042525413184156695
RMSE: 0.0240
0.023982449330589007
RMSE: 0.0731
0.0730507089627558
RMSE: 0.0177
0.017732792232917058
RM

RMSE: 0.0088
0.008767716209688225
RMSE: 0.0103
0.010293960620665569
Predicting....
('Playlist Size:', 1865)
Writing to file.....
(1950, u'sad girl')
Finding Similar Playlists...
Creating Rating matrix...
Loading data for training
Training....
RMSE: 0.0088
0.008822437754340103
RMSE: 0.0413
0.041286213787535934
RMSE: 0.1287
0.12865635657207253
RMSE: 0.0194
0.01944128230643296
RMSE: 0.0094
0.009397998879628966
RMSE: 0.0119
0.011927720034868677
RMSE: 0.0638
0.06378229941842718
RMSE: 0.0084
0.008425485029091137
RMSE: 0.0162
0.01621859888761929
RMSE: 0.0121
0.012094418955291106
Predicting....
('Playlist Size:', 2053)
Writing to file.....
(1951, u'workout')
Finding Similar Playlists...
Creating Rating matrix...
Loading data for training
Training....
RMSE: 0.0055
0.005519591174506129
RMSE: 0.0070
0.006986585502513249
RMSE: 0.0216
0.021632664013490358
RMSE: 0.0160
0.015959166954493897
RMSE: 0.0039
0.003898345287063422
RMSE: 0.0025
0.0024798798762882997
RMSE: 0.0616
0.061564176476095855
RMSE: 0.

RMSE: 0.0601
0.06005516961114616
RMSE: 0.0109
0.010866512966501495
RMSE: 0.0263
0.02634170116000585
Predicting....
('Playlist Size:', 1877)
Writing to file.....
(1966, u'Rock')
Finding Similar Playlists...
Creating Rating matrix...
Loading data for training
Training....
RMSE: 0.0304
0.0303634200551946
RMSE: 0.0074
0.007361354251505994
RMSE: 0.0111
0.011072241805068612
RMSE: 0.0099
0.009908914639295293
RMSE: 0.0061
0.006128359274952136
RMSE: 0.0213
0.02131710087397363
RMSE: 0.0121
0.012070333867830418
RMSE: 0.0173
0.017333567215938244
RMSE: 0.0098
0.009793796363272983
RMSE: 0.0641
0.06406428726978818
Predicting....
('Playlist Size:', 5313)
Writing to file.....
(1967, u'Game Day')
Finding Similar Playlists...
Creating Rating matrix...
Loading data for training
Training....
RMSE: 0.0135
0.013518979340330485
RMSE: 0.0108
0.010786081526515306
RMSE: 0.0254
0.025365595957257486
RMSE: 0.0086
0.008628246206005738
RMSE: 0.0147
0.014659155324780508
RMSE: 0.0099
0.009887826118656866
RMSE: 0.0109
0

RMSE: 0.0077
0.0077110721762460805
RMSE: 0.0087
0.008743760147115134
Predicting....
('Playlist Size:', 3529)
Writing to file.....
(1982, u'Fall Playlist')
Finding Similar Playlists...
Creating Rating matrix...
Loading data for training
Training....
RMSE: 0.0094
0.009363454681676204
RMSE: 0.0069
0.006919016168503277
RMSE: 0.0065
0.006508512987296638
RMSE: 0.0204
0.020409331566264913
RMSE: 0.0124
0.012378482401071787
RMSE: 0.0109
0.010930047654877598
RMSE: 0.0072
0.007152581151712735
RMSE: 0.0108
0.01076742513180632
RMSE: 0.0078
0.007767558306415255
RMSE: 0.0148
0.014822503290192732
Predicting....
('Playlist Size:', 2875)
Writing to file.....
(1983, u'fire\U0001f525\U0001f525')
Finding Similar Playlists...
Creating Rating matrix...
Loading data for training
Training....
RMSE: 0.0088
0.008814930326116185
RMSE: 0.0075
0.007463963077200252
RMSE: 0.0111
0.011083479364086504
RMSE: 0.0062
0.006209767121588789
RMSE: 0.0106
0.01055395190283077
RMSE: 0.0334
0.03337341096199239
RMSE: 0.0079
0.0078

RMSE: 0.0849
0.08490248213332209
Predicting....
('Playlist Size:', 1157)
Writing to file.....
(1998, u'Main')
Finding Similar Playlists...
Creating Rating matrix...
Loading data for training
Training....
RMSE: 0.0755
0.07551166322880108
RMSE: 0.1561
0.15612110143214172
RMSE: 0.0148
0.01477983225895665
RMSE: 0.0328
0.032824984763362056
RMSE: 0.0168
0.01677367192533564
RMSE: 0.0489
0.04891036905664163
RMSE: 0.0157
0.015748394127083752
RMSE: 0.0141
0.01410718958252089
RMSE: 0.0080
0.008010014789830594
RMSE: 0.0112
0.01118027023042347
Predicting....
('Playlist Size:', 1438)
Writing to file.....
(1999, u'\U0001f483\U0001f3fb')
Finding Similar Playlists...
Creating Rating matrix...
Loading data for training
Training....
RMSE: 0.1941
0.1941434538758033
RMSE: 0.0378
0.037751563687896804
RMSE: 0.0174
0.01744026147642382
RMSE: 0.0159
0.015934650781930237
RMSE: 0.0386
0.03860105517145951
RMSE: 0.0172
0.01717369232538069
RMSE: 0.0952
0.0952123952941169
RMSE: 0.0473
0.047326215364173155
RMSE: 0.062